<a href="https://colab.research.google.com/github/RocioLiu/AI-for-Trading/blob/main/Character_Level_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Character-Level LSTM in PyTorch**
In this notebook, I'll construct a character-level LSTM with PyTorch. The network will train character by character on some text, then generate new text character by character. As an example, I will train on Anna Karenina. **This model will be able to generate new text based on the text from the book!**

This network is based off of Andrej Karpathy's [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [implementation in Torch](https://github.com/karpathy/char-rnn). Below is the general architecture of the character-wise RNN.

![](https://raw.githubusercontent.com/udacity/deep-learning-v2-pytorch/7114a996e4edbb9570e57d3afa8848e8b0ebe6e0/recurrent-neural-networks/char-rnn/assets/charseq.jpeg)

First let's load in our required resources for data loading and model creation.

In [76]:
import numpy as np
import os
import torch
from torch import nn
import torch.nn.functional as F

## **Load in Data**
Then, we'll load the Anna Karenina text file and convert it into integers for our network to use.

In [77]:
from google.colab import files
uploaded = files.upload()

In [85]:
# open text file and read in data as `text`
with open('data/anna.txt', 'r') as f:
    text = f.read()


Let's check out the first 100 characters, make sure everything is peachy. According to the [American Book Review](http://americanbookreview.org/100bestlines.asp), this is the 6th best first line of a book ever.

In [86]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

### **Tokenization**
In the cells, below, I'm creating a couple of **dictionaries** to convert the characters to and from integers. Encoding the characters as integers makes it easier to use as input in the network.

In [87]:
# encode the text and map each character to an integer and vice versa

# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: i for i, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])

And we can see those same characters from above, encoded as integers.

In [88]:
encoded[:100]

array([54, 22, 34, 21, 20, 78, 23, 74, 79, 82, 82, 82, 12, 34, 21, 21, 40,
       74, 44, 34, 33, 29, 27, 29, 78, 31, 74, 34, 23, 78, 74, 34, 27, 27,
       74, 34, 27, 29, 75, 78,  6, 74, 78, 14, 78, 23, 40, 74, 45, 71, 22,
       34, 21, 21, 40, 74, 44, 34, 33, 29, 27, 40, 74, 29, 31, 74, 45, 71,
       22, 34, 21, 21, 40, 74, 29, 71, 74, 29, 20, 31, 74, 49, 57, 71, 82,
       57, 34, 40, 67, 82, 82, 53, 14, 78, 23, 40, 20, 22, 29, 71])

## **Pre-processing the data**
As you can see in our char-RNN image above, our LSTM expects an input that is **one-hot encoded** meaning that each character is converted into an integer (via our created dictionary) and then converted into a column vector where only it's corresponding integer index will have the value of 1 and the rest of the vector will be filled with 0's. Since we're one-hot encoding the data, let's make a function to do that!

In [89]:
def one_hot_encoded(arr, n_labels):

    # Initialize the encoded array
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)

    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.

    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))

    return one_hot

In [90]:
# check that the function works as expected
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encoded(test_seq, 8)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


## **Making training mini-batches**
To train on this data, we also want to create mini-batches for training. Remember that we want our batches to be multiple sequences of some desired number of sequence steps. Considering a simple example, our batches would look like this:

[](https://raw.githubusercontent.com/udacity/deep-learning-v2-pytorch/7114a996e4edbb9570e57d3afa8848e8b0ebe6e0/recurrent-neural-networks/char-rnn/assets/sequence_batching%401x.png)

In this example, we'll take the encoded characters (passed in as the arr parameter) and split them into multiple sequences, given by `batch_size`. Each of our sequences will be `seq_length` long.

### **Creating Batches**
**1. The first thing we need to do is discard some of the text so we only have completely full mini-batches.**

Each batch contains $N \times M$ characters, where $N$ is the batch size (the number of sequences in a batch) and $M$ is the seq_length or number of time steps in a sequence. Then, to get the total number of batches, $K$, that we can make from the array `arr`, you divide the length of `arr` by the number of characters per batch. Once you know the number of batches, you can get the total number of characters to keep from arr, $N * M * K$.

**2. After that, we need to split arr into $N$ batches.**

You can do this using `arr.reshape(size)` where size is a tuple containing the dimensions sizes of the reshaped array. We know we want $N$ sequences in a batch, so let's make that the size of the first dimension. For the second dimension, you can use -1 as a placeholder in the size, it'll fill up the array with the appropriate data for you. After this, you should have an array that is $N \times (M * K)$.

**3. Now that we have this array, we can iterate through it to get our mini-batches.**

The idea is each batch is a $N \times M$ window on the $N \times (M * K)$ array. For each subsequent batch, the window moves over by `seq_length`. We also want to create both the input and target arrays. Remember that the targets are just the inputs shifted over by one character. The way I like to do this window is use range to take steps of size `n_steps` from $0$ to `arr.shape[1]`, the total number of tokens in each sequence. That way, the integers you get from range always point to the start of a batch, and each window is `seq_length` wide.

**TODO:** Write the code for creating batches in the function below. The exercises in this notebook will not be easy. I've provided a notebook with solutions alongside this notebook. If you get stuck, checkout the solutions. The most important thing is that you don't copy and paste the code into here, type out the solution code yourself.

In [75]:
10//3

3

In [65]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    ## TODO: Get the number of batches we can make
    n_batches = arr // (batch_size * seq_length)
    
    ## TODO: Keep only enough characters to make full batches
    arr = arr[:(n_batches * batch_size * seq_length)]
    
    ## TODO: Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    
    ## TODO: Iterate over the batches using a window of size seq_length
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = 
        # The targets, shifted by one
        y = 
        yield x, y

In [92]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'